In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
movies = pd.read_csv('movies.csv')

In [3]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [5]:
movies.shape

(4803, 24)

In [6]:
movies = movies[['index','original_title','genres','keywords','overview','tagline','cast','director']]

In [7]:
movies.head()

,index,original_title,genres,keywords,overview,tagline,cast,director
0,0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,2,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,3,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,4,John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [8]:
movies.fillna('')

,index,original_title,genres,keywords,overview,tagline,cast,director
0,0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,2,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,3,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,4,John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...,...,...
4798,4798,El Mariachi,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,El Mariachi just wants to play his guitar and ...,"He didn't come looking for trouble, but troubl...",Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,4799,Newlyweds,Comedy Romance,,A newlywed couple's honeymoon is upended by th...,A newlywed couple's honeymoon is upended by th...,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,4800,"Signed, Sealed, Delivered",Comedy Drama Romance TV Movie,date love at first sight narration investigati...,"""Signed, Sealed, Delivered"" introduces a dedic...",,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,4801,Shanghai Calling,,,When ambitious New York attorney Sam is sent t...,A New Yorker in Shanghai,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [9]:
combine_fetures = movies['genres']+movies['keywords']+movies['tagline']+movies['director']+movies['cast']

In [10]:
#converting text data into vectors
vectorizer = TfidfVectorizer()


In [11]:
feature_vector = vectorizer.fit_transform(combine_fetures.apply(lambda x: np.str_(x)))

In [12]:
print(feature_vector)

  (0, 17037)	0.2092633083058011
  (0, 13586)	0.1608978254806352
  (0, 11842)	0.23519260716139767
  (0, 19248)	0.15479777183310411
  (0, 21994)	0.1891643163486044
  (0, 18538)	0.21153118581074126
  (0, 17655)	0.21153118581074126
  (0, 22820)	0.20332167956056799
  (0, 22532)	0.22278442527498424
  (0, 2948)	0.26881271608777757
  (0, 10807)	0.11858668733641127
  (0, 15228)	0.25640453420136405
  (0, 14888)	0.08826203981382355
  (0, 22512)	0.12668347765087568
  (0, 20038)	0.07574227223605329
  (0, 18810)	0.26881271608777757
  (0, 3873)	0.2476007890478111
  (0, 21818)	0.13395678681302664
  (0, 18941)	0.338214663461617
  (0, 8332)	0.16422227543015544
  (0, 3680)	0.21956014674697863
  (0, 7587)	0.26881271608777757
  (0, 17980)	0.10160666684261595
  (0, 7270)	0.11892734772189206
  (0, 382)	0.09291659462084569
  :	:
  (4798, 9086)	0.17861816925382282
  (4798, 20865)	0.321436390874158
  (4798, 10796)	0.16288197728341544
  (4798, 13062)	0.1652739271419625
  (4798, 12054)	0.18368409155563786
  (4798

In [13]:
#getting similarity score 

similarity = cosine_similarity(feature_vector)

In [14]:
similarity

array([[1.        , 0.07039641, 0.01590931, ..., 0.        , 0.        ,
        0.        ],
       [0.07039641, 1.        , 0.02868165, ..., 0.        , 0.        ,
        0.        ],
       [0.01590931, 0.02868165, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ]])

In [15]:
similarity.shape

(4803, 4803)

In [16]:
similarity[0]

array([1.        , 0.07039641, 0.01590931, ..., 0.        , 0.        ,
       0.        ])

In [17]:
#getting movie name from user
movie_name = input('Enter Your Favorite Movie Name : ')

Enter Your Favorite Movie Name : iron man


In [18]:
#creating the list of all the movie name
list_of_all_title = movies['original_title'].tolist()
print(list_of_all_title)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [19]:
#finding close match to this movie name
find_close_match = difflib.get_close_matches(movie_name,list_of_all_title)

In [20]:
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [21]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [22]:
#finding index of movies with title
index_of_movie = movies[movies.original_title == close_match]['index'].values[0]
print(index_of_movie)

68


In [23]:
#getting list of similar movies
similarity_score = list(enumerate(similarity[index_of_movie]),)
print(similarity_score)

[(0, 0.016165365891025286), (1, 0.035030909548490016), (2, 0.006603856220832992), (3, 0.007081075103521756), (4, 0.015859675761687828), (5, 0.0067406868592295305), (6, 0.021705361104677343), (7, 0.13097414519846917), (8, 0.0), (9, 0.035960300111282845), (10, 0.0), (11, 0.005848675788260867), (12, 0.0), (13, 0.006283986144435082), (14, 0.04563656313153945), (15, 0.0), (16, 0.17268116147425733), (17, 0.006303815144529588), (18, 0.0158803908074938), (19, 0.005580687451986134), (20, 0.06921008381003509), (21, 0.005671785972676326), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.14603298839984716), (27, 0.014703401713991713), (28, 0.059647333470163766), (29, 0.006844477796333289), (30, 0.07188764509085524), (31, 0.24477508631817263), (32, 0.020121567798450295), (33, 0.12640015295482818), (34, 0.0), (35, 0.03472038619212561), (36, 0.03103474261098656), (37, 0.0), (38, 0.10203478900248697), (39, 0.07791532316968015), (40, 0.0), (41, 0.015427840491792208), (42, 0.0), (43, 0.061216992486871

In [24]:
#sorting the movies base on similarity score
sorted_similar_movies = sorted(similarity_score,key= lambda x:x[1],reverse=True)

In [25]:
print(sorted_similar_movies)

[(68, 1.0), (79, 0.2988054501163676), (31, 0.24477508631817263), (16, 0.17268116147425733), (26, 0.14603298839984716), (7, 0.13097414519846917), (33, 0.12640015295482818), (783, 0.12130442597323066), (46, 0.12130323302239848), (1553, 0.1143936285059215), (511, 0.11423900056549742), (203, 0.1120574268783198), (174, 0.1092761716326641), (607, 0.1088749022145703), (138, 0.10500439773475895), (353, 0.103335260482958), (64, 0.10314662614819706), (38, 0.10203478900248697), (101, 0.10075610040042274), (1740, 0.09444716379314898), (800, 0.09399168747408733), (788, 0.09341570548537531), (1406, 0.09154866373779386), (720, 0.0894163498065788), (1210, 0.08876935901308644), (2235, 0.08705385783409236), (2186, 0.08633610777944176), (85, 0.08539590033926522), (1282, 0.08535525622947379), (182, 0.08522023597306659), (307, 0.08521319689980977), (2651, 0.08382628108902154), (3443, 0.08360601648872056), (4343, 0.0827287434058352), (2880, 0.08250785647805814), (2066, 0.08060060732244316), (4033, 0.0800743

In [26]:
#print the name or similar movies base on index
print('Movies Suggested For You : \n')

i=1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_movies = movies[movies.index == index]['original_title'].values[0]
    if (i<21):
        print(i , '.',title_from_movies)
        i+=1

Movies Suggested For You : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . The Avengers
5 . Captain America: Civil War
6 . Avengers: Age of Ultron
7 . X-Men: The Last Stand
8 . Mortdecai
9 . X-Men: Days of Future Past
10 . Se7en
11 . X-Men
12 . X2
13 . The Incredible Hulk
14 . Sky Captain and the World of Tomorrow
15 . The Last Airbender
16 . Tropic Thunder
17 . X-Men: Apocalypse
18 . The Amazing Spider-Man 2
19 . X-Men: First Class
20 . Kick-Ass 2


# Movie Recommendation System

In [31]:
#combining all codes and making recommandation system

movie_name = input('Enter Your Favorite Movie Name : ')

list_of_all_title = movies['original_title'].tolist()

find_close_match = difflib.get_close_matches(movie_name,list_of_all_title)

close_match = find_close_match[0]

index_of_movie = movies[movies.original_title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_movie]),)

sorted_similar_movies = sorted(similarity_score,key= lambda x:x[1],reverse=True)

print('Movies Suggested For You : \n')

i=1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_movies = movies[movies.index == index]['original_title'].values[0]
    if (i<21):
        print(i , '.',title_from_movies)
        i+=1

Enter Your Favorite Movie Name : anaconda
Movies Suggested For You : 

1 . Anaconda
2 . Do the Right Thing
3 . Angel Eyes
4 . Ice Age: The Meltdown
5 . The Mighty Ducks
6 . Alien
7 . The Shawshank Redemption
8 . Bound
9 . Blades of Glory
10 . The Life Aquatic with Steve Zissou
11 . Capitalism: A Love Story
12 . National Treasure
13 . Midnight Cowboy
14 . Logan's Run
15 . Three Kings
16 . Enemy of the State
17 . Pride and Glory
18 . Harvard Man
19 . Sorstalanság
20 . Martha Marcy May Marlene
